In [2]:
import pandas as pd
import numpy as np
import joblib

# Models
from keras.models import Sequential
from keras.layers import Dense

#Tuning and Cross Validation
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

#### Importing Data

In [ ]:
x_train = pd.read_csv('../../Data Files/Training Data/x_train.csv')
x_test = pd.read_csv('../../Data Files/Training Data/x_test.csv')
y_train = pd.read_csv('../../Data Files/Training Data/y_train.csv')
y_test = pd.read_csv('../../Data Files/Training Data/y_test.csv')

#### Defining Neural Network Architecture

In [ ]:
def build_nn_model(num_neurons_1, num_neurons_2):
    nn = Sequential()
    nn.add(Dense(num_neurons_1, input_shape=[len(x_train.columns),], activation="relu"))
    nn.add(Dense(num_neurons_2, input_shape=[len(x_train.columns),], activation="relu"))
    nn.add(Dense(1, activation='linear'))
    nn.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse']) 
    return nn

#### Training Neural Network

In [ ]:
nn = build_nn_model(30, 30)
nn_history = nn.fit(x_train, y_train, batch_size=5, epochs=5, verbose=1)
y_pred_nn = nn.predict(x_test)

#### Tuning for Hyperparameters

In [ ]:
model = KerasRegressor(build_fn=build_nn_model)

param_grid = {'activation': ['relu', 'softmax'],
              'optimizer': ['adam', 'sgd'],
              'batch_size': [5, 10, 20],
              'epochs': [5, 10, 20]}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(x_train, y_train)

optimal_clf = grid_search.best_estimator_

Y_pred = optimal_clf.predict(x_test)
mse_after_tuning = mean_squared_error(y_test, Y_pred)
print("test accuracy = {:.2%}".format(mse_after_tuning))

#### Cross Validation

In [ ]:
scores = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error:", -scores.mean())
print("Standard deviation:", scores.std())

#### Saving Model File and Predictions

In [ ]:
save_path = '../../Data Files/Model Files/'
nn.save(save_path + 'nn.h5')
joblib.dump(y_pred_nn, save_path + 'neuralnetwork_output.csv')